# Data Modeling with PostgreSQL

Processing subset of millionsongs dataset using Postgres.

Sample data inside github repo (https://github.com/gyhou/millionsongs).

# ETL Processes
This notebook was used to verify the ETL process for each of the tables inside the `etl.py` file

In [0]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [0]:
import create_tables
create_tables.main()

In [0]:
host = ''
dbname = ''
user = '' 
password = ''
conn = psycopg2.connect(f"host={host} dbname={dbname} user={user} password={password}")
cur = conn.cursor()

In [0]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
Perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables, and load records into each table.

- Use the `get_files` function above to get a list of all song JSON files in `data/song_data`
- Read the song file and view the data

In [0]:
song_files = get_files('data/song_data')

In [0]:
# Combine all songs dataset
df_list = []
for filepath in song_files:
    df = pd.read_json(filepath, lines=True)
    df_list.append(df)
df = pd.concat(df_list)
print(df.shape)
df.head()

(72, 10)


,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,AR8IEZO1187B99055E,NaN,,NaN,Marc Shaiman,149.86404,1,SOINLJW12A8C13314C,City Slickers,2008
0,AR558FS1187FB45658,NaN,,NaN,40 Grit,75.67628,1,SOGDBUF12A8C140FAA,Intro,2003
0,ARVBRGZ1187FB4675A,NaN,,NaN,Gwen Stefani,290.55955,1,SORRZGD12A6310DBC3,Harajuku Girls,2004
0,ARNF6401187FB57032,40.79086,"New York, NY [Manhattan]",-73.96644,Sophie B. Hawkins,305.16200,1,SONWXQJ12A8C134D94,The Ballad Of Sleeping Beauty,1994
0,AREVWGE1187B9B890A,-13.44200,Noci (BA),-41.99520,Bitter End,282.43546,1,SOFCHDR12AB01866EF,Living Hell,0


### Dealing with null values

In [0]:
df.isnull().sum()

artist_id            0
artist_latitude     41
artist_location      0
artist_longitude    41
artist_name          0
duration             0
num_songs            0
song_id              0
title                0
year                44
loc                 28
location             0
dtype: int64

In [0]:
# Replacing empty locations with NaN
import numpy as np
df['artist_location'].replace('', np.nan, inplace=True)
df.head()

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year,loc,location
0,AR8IEZO1187B99055E,NaN,NaN,NaN,Marc Shaiman,149.86404,1,SOINLJW12A8C13314C,City Slickers,2008.0,NaN,False
0,AR558FS1187FB45658,NaN,NaN,NaN,40 Grit,75.67628,1,SOGDBUF12A8C140FAA,Intro,2003.0,NaN,False
0,ARVBRGZ1187FB4675A,NaN,NaN,NaN,Gwen Stefani,290.55955,1,SORRZGD12A6310DBC3,Harajuku Girls,2004.0,NaN,False
0,ARNF6401187FB57032,40.79086,"New York, NY [Manhattan]",-73.96644,Sophie B. Hawkins,305.16200,1,SONWXQJ12A8C134D94,The Ballad Of Sleeping Beauty,1994.0,"New York, NY [Manhattan]",False
0,AREVWGE1187B9B890A,-13.44200,Noci (BA),-41.99520,Bitter End,282.43546,1,SOFCHDR12AB01866EF,Living Hell,NaN,Noci (BA),False


In [0]:
df.isnull().sum()

## #1: `songs` Table
#### Extract Data for Songs Table
- Selecting columns song ID, title, artist ID, year, and duration for `songs` table
- Use `df.values` to select just the values from the dataframe
- Convert the array to a list and set it to `song_data`

In [0]:
# Index to select the first (only) record in the dataframe
song_data = df[['song_id','title','artist_id','year','duration']].values[0]
song_data

array(['SOINLJW12A8C13314C', 'City Slickers', 'AR8IEZO1187B99055E', 2008,
       149.86404], dtype=object)

#### Insert Record into Song Table
- The `song_table_insert` query in `sql_queries.py` will insert data into the `songs` table
- Run the cell below to insert a record for this song into the `songs` table
- Run `create_tables.py` before running the cell below to create/reset the `songs` table in the sparkify database.

In [0]:
cur.execute(song_table_insert, song_data)
conn.commit()

Run `test.ipynb` to see if data was successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Selecting columns artist ID, name, location, latitude, and longitude for `artists` table
- Use `df.values` to select just the values from the dataframe
- Convert the array to a list and set it to `artist_data`

In [0]:
# Index to select the first (only) record in the dataframe
artist_data = df[['artist_id','artist_name','artist_location','artist_latitude','artist_longitude']].values[0]
artist_data

array(['AR8IEZO1187B99055E', 'Marc Shaiman', '', nan, nan], dtype=object)

#### Insert Record into Artist Table
- The `artist_table_insert` query in `sql_queries.py` will insert data into the `artists` table
- Run the cell below to insert a record for this song's artist into the `artists` table
- Make sure `artists` table was created/resetted in the sparkify database by `create_tables.py` before running the cell below

In [0]:
cur.execute(artist_table_insert, artist_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
Perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, the `songplays` fact table, then insert records into each table.

- Use the `get_files` function above to get a list of all log JSON files in `data/log_data`
- Read the log file and view the data

In [0]:
log_files = get_files('data/log_data')

In [0]:
# The first log file in this list
filepath_0 = log_files[0]
df_0 = pd.read_json(filepath_0, lines=True)
print(df_0.shape)
df_0.head(1)

(366, 18)


,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Sydney Youngblood,Logged In,Jacob,M,53,Klein,238.07955,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,954,Ain't No Sunshine,200,1543449657796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73


In [0]:
# Combine all logs dataset
df_list = []
for filepath in log_files:
    df = pd.read_json(filepath, lines=True)
    df_list.append(df)
df = pd.concat(df_list)
print(df.shape)
df.head()

(8056, 18)


,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Sydney Youngblood,Logged In,Jacob,M,53,Klein,238.07955,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,954,Ain't No Sunshine,200,1543449657796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
1,Gang Starr,Logged In,Layla,F,88,Griffin,151.92771,paid,"Lake Havasu City-Kingman, AZ",PUT,NextSong,1.541057e+12,984,My Advice 2 You (Explicit),200,1543449690796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",24
2,3OH!3,Logged In,Layla,F,89,Griffin,192.52200,paid,"Lake Havasu City-Kingman, AZ",PUT,NextSong,1.541057e+12,984,My First Kiss (Feat. Ke$ha) [Album Version],200,1543449841796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",24
3,RÃÂ¶yksopp,Logged In,Jacob,M,54,Klein,369.81506,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,954,The Girl and The Robot,200,1543449895796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
4,Kajagoogoo,Logged In,Layla,F,90,Griffin,223.55546,paid,"Lake Havasu City-Kingman, AZ",PUT,NextSong,1.541057e+12,984,Too Shy,200,1543450033796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",24


## Check for logs where users are listening to songs
- Filter records by `NextSong` action
- Also check for null values

In [0]:
df = df[df['page']=='NextSong']
df.isnull().sum()

artist           0
auth             0
firstName        0
gender           0
itemInSession    0
lastName         0
length           0
level            0
location         0
method           0
page             0
registration     0
sessionId        0
song             0
status           0
ts               0
userAgent        0
userId           0
dtype: int64

## #3: `time` Table
#### Extract Data for Time Table
- Selecting timestamp, hour, day, week of year, month, year, and weekday from the `ts` column for `time_data` table

In [0]:
# Show only logs where users are listening to a song
df = df[df['page']=='NextSong']
print(df.shape)
df.head(1)

(319, 18)


,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Sydney Youngblood,Logged In,Jacob,M,53,Klein,238.07955,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,954,Ain't No Sunshine,200,1543449657796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73


In [0]:
# convert timestamp column to datetime
df['datetime'] = pd.to_datetime(df['ts'], unit='ms')
df['hour'] = df['datetime'].dt.hour
df['day'] = df['datetime'].dt.day
df['week'] = df['datetime'].dt.week
df['month'] = df['datetime'].dt.month
df['year'] = df['datetime'].dt.year
df['weekday'] = df['datetime'].dt.weekday
df.head(1)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId,datetime,hour,day,week,month,year,weekday
0,Sydney Youngblood,Logged In,Jacob,M,53,Klein,238.07955,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,954,Ain't No Sunshine,200,1543449657796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73,2018-11-29 00:00:57.796,0,29,48,11,2018,3


In [0]:
# set time_data to a list containing these values in order
time_data = list(zip(list(df['ts']), list(t.dt.hour), list(t.dt.day), list(t.dt.week), list(t.dt.month), list(t.dt.year), list(t.dt.weekday)))
# Specify labels for these columns and set to column_labels
column_labels = ['start_time', 'hour', 'day', 'week', 'month', 'year', 'weekday']

Create a dataframe, `time_df,` containing the time data for this file

In [0]:
time_df = pd.DataFrame(time_data, columns=column_labels, dtype=int)
print(time_df.shape)
time_df.head()

(319, 7)


,start_time,hour,day,week,month,year,weekday
0,1543449657796,0,29,48,11,2018,3
1,1543449690796,0,29,48,11,2018,3
2,1543449841796,0,29,48,11,2018,3
3,1543449895796,0,29,48,11,2018,3
4,1543450033796,0,29,48,11,2018,3


In [0]:
time_df = df[['ts', 'hour', 'day', 'week', 'month', 'year', 'weekday']]
time_df = time_df.rename(columns={'ts':'start_time'})
print(time_df.shape)
time_df.head(1)

(319, 7)


,start_time,hour,day,week,month,year,weekday
0,1543449657796,0,29,48,11,2018,3


### Data Quality Check
- Set Data types if needed
- Remove Duplicates

In [0]:
time_df.dtypes

start_time    int64
hour          int64
day           int64
week          int64
month         int64
year          int64
weekday       int64
dtype: object

In [0]:
time_df.duplicated('start_time').value_counts()

False    319
dtype: int64

In [0]:
# Drop all duplicates on column 'start_time'
time_df = time_df.drop_duplicates(subset=['start_time'], keep=False)
time_df.shape

(319, 7)

#### Insert Records into Time Table
- The `time_table_insert` query in `sql_queries.py` will insert data into the `time` table
- Make sure `time` table was created/resetted in the sparkify database by `create_tables.py` before running the cell below

In [0]:
!pip install psycopg2-binary

In [0]:
# Append entire dataframe to postgres database table

from sqlalchemy import create_engine
url = f"postgres://{user}:{password}@{host}:5432/{dbname}"
engine = create_engine(url)
time_df.to_sql(name="time", con=engine, if_exists="append", index=False)

Run the cell below to insert records by row for the timestamps in this log file into the `time` table

In [0]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Selecting columns user ID, first name, last name, gender and level for `users` table

In [0]:
df["userId"] = pd.to_numeric(df["userId"])
user_df = df[['userId','firstName','lastName','gender','level']]
user_df = user_df.rename(columns={'userId':'user_id','firstName':'first_name','lastName':'last_name'})
print(user_df.shape)
user_df.head(2)

(319, 5)


,user_id,first_name,last_name,gender,level
0,73,Jacob,Klein,M,paid
1,24,Layla,Griffin,F,paid


In [0]:
user_df.dtypes

user_id        int64
first_name    object
last_name     object
gender        object
level         object
dtype: object

#### Insert Records into Users Table

- The `user_table_insert` query in sql_queries.py will insert data into the `users` table
- Make sure `users` table was created/resetted in the sparkify database by `create_tables.py` before running the cell below


**Data Quality Check** - remove duplicate user id

In [0]:
# Drop duplicates except for the last occurrence on column 'userId'
user_df_dd = user_df.drop_duplicates(subset=['user_id'], keep='last')
print(user_df_dd.shape)
user_df_dd

(27, 5)


,user_id,first_name,last_name,gender,level
7,73,Jacob,Klein,M,paid
21,50,Ava,Robinson,F,free
34,24,Layla,Griffin,F,paid
39,54,Kaleb,Cook,M,free
47,14,Theodore,Harris,M,free
49,98,Jordyn,Powell,F,free
53,78,Chloe,Roth,F,free
55,101,Jayden,Fox,M,free
56,10,Sylvie,Cruz,F,free
57,52,Theodore,Smith,M,free


In [0]:
# Append entire dataframe to postgres database table
# url = f"postgres://{user}:{password}@{host}:5432/{dbname}"
# engine = create_engine(url)
# user_df_dd.to_sql(name="users", con=engine, if_exists="append", index=False)

In [0]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
Since the log file does not specify an ID for either the song or the artist, we got the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- The `song_select` query in `sql_queries.py` will find the song ID and artist ID based on the title, artist name, and duration of a song
- Selecting columns timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent for `songplay_data` table

#### Insert Records into Songplays Table
- The `songplay_table_insert` query will insert records for the songplay actions in this log file into the `songplays` table
- Make sure `songplays` table was created/resetted in the sparkify database by `create_tables.py` before running the cell below

In [0]:
df.head(1)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId,datetime,hour,day,week,month,year,weekday
0,Sydney Youngblood,Logged In,Jacob,M,53,Klein,238.07955,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,954,Ain't No Sunshine,200,1543449657796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73,2018-11-29 00:00:57.796,0,29,48,11,2018,3


In [0]:
for index, row in df.iterrows():

    # Get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    
    # If song id and artist id not found, set as 'None'
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None

    # songplay_table_insert - imported from sql_queries.py
    songplay_data = (row.ts, row.userId, row.level, songid, artistid, row.sessionId, row.location, row.userAgent)
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [0]:
conn.close()